# Sample Loading

Notebook to load Sample data into the databse

## Imports

In [96]:
import psycopg2
from tabulate import tabulate
#import a folder in the parent directory
import sys
sys.path.append('../')
import queries.queries as qrs

## Connection

In [ ]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connected to the database


## Panel id search

A panel is made of a material, so the panel table is linked to the material table.

Here we have to check the material table and to set the meterial of the panel.

In [ ]:
parent_table_name = 'panels'

parent_id_column = parent_table_name[:-1] + '_id'

parent_data = qrs.get_data(parent_table_name)

parent_data

,id_panel,name_panel,material_id_panel
0,1,Juan Ignacio,1


In [99]:
#we set the id
parent_id = 1

## Sample data

We have to add the parameters of the sample.

The name is a compulsory parameter, it has to be always filled.

The panel_id(parent_id_column) parameter is chosen in the previous cells

In [100]:
name = 'JI_12'

main_parameters = {'name':name, parent_id_column:parent_id} 


Then metadata parameters are optional and can vary depending on the sample so its loading to the database is more dinamic.

The metadata list can be empty.

To load metadata parameters we have to fill a list with dictionaries, the dictionaries may have this parameters:

1. key: The name of the parameter

2. value: The value of the parameter

3. type: The units of the parameter

Here we include an example to save the height, which units are millimeters and the value is 200:

metadata_parameteres.append({'key':'height', 'value':'200', 'type':'mm'})

In [ ]:
#metadata

metadata_parameters = []

metadata_parameters.append({'key':'geometry_type', 'value':'parallel', 'type':'nominal'})

metadata_parameters.append({'key':'height', 'value':150, 'type':'mm'})

metadata_parameters.append({'key':'width', 'value':40, 'type':'mm'})

metadata_parameters.append({'key':'thickness', 'value':5, 'type':'mm'})

metadata_parameters.append({'key':'keyholes', 'value':'True', 'type':'Boolean'})

## Load into the table

In [102]:
print('Parameters to be inserted: ')
for key, value in main_parameters.items():
    print(f"    -- {key}: {value}")

Parameters to be inserted: 
    -- name: JI_12
    -- panel_id: 1


In [103]:
table_name = 'samples'

# Extract column names and values from the attributes dictionary
columns = ', '.join(main_parameters.keys())
values = ', '.join([f"'{v}'" for v in main_parameters.values()])

# Construct the SQL INSERT statement
sql = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"

print(sql)

# Create a cursor object using the cursor() method
cursor = conn.cursor()

# Execute the SQL statement
cursor.execute(sql)

cursor.execute('COMMIT')

cursor.close()

INSERT INTO samples (name, panel_id) VALUES ('JI_12', '1')


## Make sure insert was correct

In [ ]:
data = qrs.data_parent(table_name,parent_table_name,parent_id_column)

data

,id_sample,name_sample,geometry_type_sample,height_sample,width_sample,thickness_sample,id_panel,name_panel,material_id_panel
0,1,JI_4,parallel nominal,150 mm,40 mm,5 mm,1,Juan Ignacio,1
1,2,JI_5,parallel nominal,150 mm,40 mm,5 mm,1,Juan Ignacio,1
2,3,JI_7,parallel nominal,150 mm,40 mm,5 mm,1,Juan Ignacio,1
3,4,JI_8,parallel nominal,150 mm,40 mm,5 mm,1,Juan Ignacio,1
4,5,JI_9,parallel nominal,150 mm,40 mm,5 mm,1,Juan Ignacio,1
5,6,JI_10,parallel nominal,150 mm,40 mm,5 mm,1,Juan Ignacio,1
6,7,JI_11,parallel nominal,150 mm,40 mm,5 mm,1,Juan Ignacio,1
7,8,JI_12,NaN,NaN,NaN,NaN,1,Juan Ignacio,1


Now we store the id of the inserted row, to insert its related metadata

In [105]:
row_id = data['id_' + table_name[:-1]].values[-1]

print(f"Row ID: {row_id}")

Row ID: 8


## Load into the metadata table

In [106]:
metadata_table_name =  table_name[:-1] + '_metadata'

for attributes in metadata_parameters:

    attributes[table_name[:-1] + '_id'] = row_id

    # Extract column names and values from the attributes dictionary
    columns = ', '.join(attributes.keys())
    values = ', '.join([f"'{v}'" for v in attributes.values()])

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {metadata_table_name} ({columns}) VALUES ({values})"

    print(sql)

    # Create a cursor object using the cursor() method
    cursor = conn.cursor()

    # Execute the SQL statement
    cursor.execute(sql)

    cursor.execute('COMMIT')

    cursor.close()

INSERT INTO sample_metadata (key, value, type, sample_id) VALUES ('geometry_type', 'parallel', 'nominal', '8')
INSERT INTO sample_metadata (key, value, type, sample_id) VALUES ('height', '150', 'mm', '8')
INSERT INTO sample_metadata (key, value, type, sample_id) VALUES ('width', '40', 'mm', '8')
INSERT INTO sample_metadata (key, value, type, sample_id) VALUES ('thickness', '5', 'mm', '8')


## Make sure insert was correct

In [ ]:
data = qrs.get_data_metadata(table_name)

data

,id_sample,name_sample,panel_id_sample,geometry_type_sample,height_sample,width_sample,thickness_sample
0,1,JI_4,1,parallel nominal,150 mm,40 mm,5 mm
1,2,JI_5,1,parallel nominal,150 mm,40 mm,5 mm
2,3,JI_7,1,parallel nominal,150 mm,40 mm,5 mm
3,4,JI_8,1,parallel nominal,150 mm,40 mm,5 mm
4,5,JI_9,1,parallel nominal,150 mm,40 mm,5 mm
5,6,JI_10,1,parallel nominal,150 mm,40 mm,5 mm
6,7,JI_11,1,parallel nominal,150 mm,40 mm,5 mm
7,8,JI_12,1,parallel nominal,150 mm,40 mm,5 mm


In [108]:
conn.close()